In [55]:
import pandas as pd
import netCDF4 
import numpy as np

In [56]:
mask = netCDF4.Dataset('/pampa/picart/Masks/mask_GEM5_ERA5grid','r')

In [57]:
lon = mask.variables['lon'][:]
lat = mask.variables['lat'][:]
HU = mask.variables['HU'][:]

In [58]:
# mask to store the boundary layer
bnd = np.zeros_like(HU, dtype=bool)

In [60]:
# For each HU == True, we only want to keep the ones that have 
# at least one neighbour with HU == False 
# (Thats is how we know that the grid point is at the limit of the domain)

# On itère au travers tous les points de grille
for i in range(HU.shape[0]):
    for j in range(HU.shape[1]):
        if HU[i,j] : 
            has_false_neighbor = False
            neighbors = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]  
            for neighbor_i, neighbor_j in neighbors:
                if (neighbor_i >= 0 and neighbor_i < HU.shape[0] and
                    neighbor_j >= 0 and neighbor_j < HU.shape[1] and
                    not HU[neighbor_i, neighbor_j]):
                    has_false_neighbor = True
                    break
                if has_false_neighbor : 
                    bnd[i,j] = True
                
            
# Create a new netCDF file for the filtered data
nc_filtered = netCDF4.Dataset('/home/cloutier/crcm6_bnd.nc', 'w', format='NETCDF4')

# Define the dimensions in the new netCDF file
nc_filtered.createDimension('lon', len(lon))
nc_filtered.createDimension('lat', len(lat))

# Create variables in the new netCDF file
lon_var = nc_filtered.createVariable('lon', lon.dtype, ('lon',))
lat_var = nc_filtered.createVariable('lat', lat.dtype, ('lat',))
HU_var = nc_filtered.createVariable('HU', HU.dtype, ('lat', 'lon'))

# Assign data to the variables in the new netCDF file
lon_var[:] = lon
lat_var[:] = lat
HU_var[:] = bnd

# Save and close the new netCDF file
nc_filtered.close()

# Close the original netCDF file
# mask.close()

In [61]:
ds = netCDF4.Dataset('/home/cloutier/crcm6_bnd.nc', 'r')

In [62]:
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): lon(241), lat(161)
    variables(dimensions): float64 lon(lon), float64 lat(lat), int8 HU(lat, lon)
    groups: 